# Pytorch MNIST

Importing Modules

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


Device and Hyper parameters

In [2]:

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 128
learning_rate = 0.001
epochs = 5


Loading the MNIST dataset

In [3]:

# Load MNIST dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        root='.', train=True, download=True, transform=transforms.ToTensor()
    ),
    batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        root='.', train=False, download=True, transform=transforms.ToTensor()
    ),
    batch_size=batch_size, shuffle=False
)



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 54029163.22it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1838732.45it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 14457019.99it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4937928.66it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



Model Definition

In [4]:

# Define neural network model (equivalent to a Keras Dense model)
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)  # Input layer
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)    # Hidden layer
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 10)     # Output layer

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten the input
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer
model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


Defining Training and Testing Functions

In [5]:

# Training loop
def train(model, train_loader):
    model.train()
    for epoch in range(epochs):
        running_loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)

            # Forward pass
            outputs = model(data)
            loss = criterion(outputs, target)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Test loop
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')


Training

In [6]:
# Run training
train(model, train_loader)

Epoch [1/5], Loss: 0.4124
Epoch [2/5], Loss: 0.1745
Epoch [3/5], Loss: 0.1253
Epoch [4/5], Loss: 0.0971
Epoch [5/5], Loss: 0.0769


Testing

In [7]:
# Run testing
test(model, test_loader)

Accuracy: 97.05%
